<a href="https://colab.research.google.com/github/mataney/PyTorchCourse/blob/master/4_more_pytorch_course.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorboard

TensorBoard: TensorBoard provides the visualization and tooling needed for machine learning experimentation.

 PyTorch now (From version 1.1) natively supports TensorBoard with a simple  
 `from torch.utils.tensorboard import SummaryWriter` command.
 
 At the time of writing this tutorial there are some problems with using this (Not actively updating graphs)  
 So we will use the `torch=1.0` logging version.


## What should be mentioned:
### Scalars
### Images
### Graphs
- GEMM is General Matrix Multiplication
- Checkout the panels
  - different coloring
  - Trace inputs

### Embeddings
- PCA (In MNIST for example this will be clustered really nice)
- Better to pass your network embeddings as "features", this will be helpful to know where your network might got it wrong
  
Understanding exactly how to use TensorBoard to your advantage is a bit out of the scope, but you can find me here:

https://www.youtube.com/watch?v=qTwKwVgZEqU (From IBM Watson's Romeo Kienzler) and  
https://www.youtube.com/watch?v=XcHWLsVmHvk.  
https://www.youtube.com/watch?v=eBbEDRsCmv4.

# TorchScript

Based on: https://pytorch.org/tutorials/advanced/cpp_export.html 

Python is Awesome, but not for everything.

For example, when we want our models to be in production, sometimes the performance python offers isn't enough.

For production scenarios, C++ is very often the language of choice, even if only to bind it into other languages.
Or, to be more honest, PyTorch backend is in C++, so making our pytorch models run on C++ will be easier than other non-Python languages.

If we want to run our PyTorch module on a non-Python language we need to make it independent of Python.  
How do we do this?  
**TorchScript!**  
The core data structure in `TorchScript` is the `ScriptModule`. 


### ScriptModule

It is an analogue of torch’s nn.Module.  
In `nn.Modules` methods are implemented as Python functions, but in `ScriptModules` methods are implemented as `TorchScript` functions, a statically-typed subset of Python that contains all of PyTorch’s built-in Tensor operations. This difference allows your `ScriptModules` code to run without the need for a Python interpreter.

ScriptModules can be created in two ways:

## Converting Your PyTorch Model to TorchScript

Torch Script, a representation of a PyTorch model that can be understood, compiled and serialized by the Torch Script compiler.

There exist two ways of converting a PyTorch model to Torch Script. 
- **Tracing:**  
A mechanism in which the structure of the model is captured by evaluating it **once** using example inputs, and recording the flow of those inputs through the model. This is suitable for models that make limited use of control flow.
- **Scripting:**  
You can also add explicit annotations to your model that inform the Torch Script compiler that it may directly parse and compile your model code.

Generally speaking, **Tracing** is easier and is meant for whenever we don't have any control flows in our code. If we can't remove those control flows, **Scripting** is required

## Converting to Torch Script via Tracing

To convert a PyTorch model to Torch Script via tracing, you must pass an instance of your model along with an example input to the `torch.jit.trace` function. This will produce a `torch.jit.ScriptModule` object with the trace of your model evaluation embedded in the module’s forward method:

In [0]:
import torch
import torchvision

# Use a pre defined model without control-flows
model = torchvision.models.resnet18()

# In order to save the model it requires an example of possible imports
example = torch.rand(1, 3, 224, 224)

# Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.
traced_script_module = torch.jit.trace(model, example)

In [0]:
traced_script_module

TracedModule[ResNet](
  (conv1): TracedModule[Conv2d]()
  (bn1): TracedModule[BatchNorm2d]()
  (relu): TracedModule[ReLU]()
  (maxpool): TracedModule[MaxPool2d]()
  (layer1): TracedModule[Sequential](
    (0): TracedModule[BasicBlock](
      (conv1): TracedModule[Conv2d]()
      (bn1): TracedModule[BatchNorm2d]()
      (relu): TracedModule[ReLU]()
      (conv2): TracedModule[Conv2d]()
      (bn2): TracedModule[BatchNorm2d]()
    )
    (1): TracedModule[BasicBlock](
      (conv1): TracedModule[Conv2d]()
      (bn1): TracedModule[BatchNorm2d]()
      (relu): TracedModule[ReLU]()
      (conv2): TracedModule[Conv2d]()
      (bn2): TracedModule[BatchNorm2d]()
    )
  )
  (layer2): TracedModule[Sequential](
    (0): TracedModule[BasicBlock](
      (conv1): TracedModule[Conv2d]()
      (bn1): TracedModule[BatchNorm2d]()
      (relu): TracedModule[ReLU]()
      (conv2): TracedModule[Conv2d]()
      (bn2): TracedModule[BatchNorm2d]()
      (downsample): TracedModule[Sequential](
        (0): Tr

The function `torch.jit.trace`:

- Traces a function and return an executable ScriptModule that will be optimized using just-in-time compilation.
- `torch.jit.trace`  expects a callable a Python function or `torch.nn.Module` that will be run with example_inputs.
- example_inputs - a tuple of example inputs that will be passed to the function while tracing.


The traced ScriptModule can now be evaluated identically to a regular PyTorch module:

In [0]:
# Just for fun, let's see what we get if we feed our model a image of 1s.

output = traced_script_module(torch.ones(1, 3, 224, 224))
output[0, :5]

tensor([-0.8326, -0.5192, -0.1319, -0.6518,  0.1204], grad_fn=<SliceBackward>)

## Converting to Torch Script via Annotation

Assume the following simple model:

In [0]:
import torch

class MyModule(torch.nn.Module):
    def __init__(self, N, M):
        super(MyModule, self).__init__()
        self.weight = torch.nn.Parameter(torch.rand(N, M))

    def forward(self, input):
        if input.sum() > 0:
          output = self.weight.mv(input)
        else:
          output = self.weight + input
        return output

While simple we shouldn't use Tracing here as we have a `if` statement within our forward code.

we can convert it to a `ScriptModule` by subclassing it from `torch.jit.ScriptModule` and adding a `@torch.jit.script_method` annotation to the model’s forward method:



In [0]:
import torch

class MyModule(torch.jit.ScriptModule):
    def __init__(self, N, M):
        super(MyModule, self).__init__()
        self.weight = torch.nn.Parameter(torch.rand(N, M))

    @torch.jit.script_method
    def forward(self, input):
        if bool(input.sum() > 0):
          output = self.weight.mv(input)
        else:
          output = self.weight + input
        return output


In [0]:
my_script_module = MyModule(2, 3)

In [0]:
print(my_script_module.__class__)
print(my_script_module.__class__.__bases__)

<class '__main__.MyModule'>
(<class 'torch.jit.ScriptModule'>,)


Notice that `input.sum() > 0` returns a tensor of `tensor(0, dtype=torch.uint8)` or `tensor(1, dtype=torch.uint8)`.  
Then, using the python compiles is assigned to `False` or `True`.

Because we are not using Pytorch Compiler any more, we need to specify that `input.sum() > 0` returns a bool, so that `TorchScript` compiler will be able to compile accordingly.

## Saving

Assume we want to use the traced vesion of `resnet18`, saving is strightforward, we are still in Python

In [0]:
traced_script_module.save("model.pt")

## Loading - In C++ :shock:

This we need to do somewhere we can run C++:

- First, let's run in my mac the python script that runs the traced model.  
```
cd pytorch_course/TorchScript/python
python create_traced_model.py
```

- Now wea have a traced model saved under `model.pt`
- Download and extract TorchLib from https://pytorch.org/ (I had a bug in mac where I needed to `brew install libomp` and do [this](https://github.com/pytorch/pytorch/issues/14165))
- Source code: View `pytorch_course/TorchScript/example-app/example-app.cpp`.
- make txt file with all the dependencies: View `pytorch_course/TorchScript/example-app/CMakeLists.txt`.
- create and go to `build` directory.
- `cmake -DCMAKE_PREFIX_PATH=~/Downloads/libtorch/ ..`
- make
- `./example-app model.pt`
- Notice the outputs are the same.

# Other



## On Top of PyTorch

### AllenNLP

From allenAI: https://github.com/allenai/allennlp  
https://allennlp.org/tutorials
### Fast.AI

Started as a deep learning course for coders (http://course18.fast.ai/) resulted this repository: https://github.com/fastai/fastai

Worth knowing:  
- Example how to run on multiple GPUS [here](https://pytorch.org/tutorials/beginner/blitz/data_parallel_tutorial.html#sphx-glr-beginner-blitz-data-parallel-tutorial-py).
- A pytorch internal support for debugging bottlenecks in your program - [here](https://pytorch.org/docs/master/bottleneck.html)